In [1]:
import os
import pandas as pd

def check_files_in_directory(csv_file, directory):
    # Step 1: Read the CSV file
    df = pd.read_csv(csv_file)
    
    # Step 2: List files in the directory
    files_in_directory = os.listdir(directory)
    
    # Step 3: Compare file names
    missing_files = []
    for file_name in df['filename']:
        file_name = os.path.basename(file_name)  # Extract filename from path
        if file_name not in files_in_directory:
            missing_files.append(file_name)
    
    # Step 4: Report results
    if len(missing_files) == 0:
        print("All files described in the CSV are present in the directory.")
    else:
        print("The following files are missing from the directory:")
        for missing_file in missing_files:
            print(missing_file)

# Example usage:
csv_file = "D:\Sean\DCASE\datasets\Extract_to_Folder\TAU-urban-acoustic-scenes-2022-mobile-development\meta.csv"
directory = "D:\Sean\DCASE\datasets\Extract_to_Folder\TAU-urban-acoustic-scenes-2022-mobile-development\\audio"
check_files_in_directory(csv_file, directory)


KeyError: 'filename'